# Get Data

Import the dataset and filter the data for records from the year 2024.

In [6]:
import pandas as pd

df = pd.read_csv("Crime_Data_from_2020_to_Present.csv")
df.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,190326475,3/1/20 0:00,3/1/20 0:00,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,AA,Adult Arrest,510.0,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506
1,200106753,2/9/20 0:00,2/8/20 0:00,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,998.0,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628
2,200320258,11/11/20 0:00,11/4/20 0:00,1700,3,Southwest,356,1,480,BIKE - STOLEN,...,IC,Invest Cont,480.0,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002
3,200907217,5/10/23 0:00,3/10/20 0:00,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,IC,Invest Cont,343.0,NaN,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387
4,200412582,9/9/20 0:00,9/9/20 0:00,630,4,Hollenbeck,413,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,200 E AVENUE 28,NaN,34.0820,-118.2130


In [28]:
df['DATE OCC'] = pd.to_datetime(df['DATE OCC'], format='%Y%m%d', errors='coerce')
df['Year'] = df['DATE OCC'].dt.year
df_2024 = df[df['Year'] == 2024]
df_2024.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON,Year
877329,242108414,4/25/24 0:00,2024-04-25,1600,21,Topanga,2146,1,420,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),...,Adult Other,420.0,998.0,NaN,NaN,21400 SHERMAN WY,NaN,34.2010,-118.5977,2024
877330,240604934,1/21/24 0:00,2024-01-21,1510,6,Hollywood,668,2,624,BATTERY - SIMPLE ASSAULT,...,Invest Cont,624.0,NaN,NaN,NaN,1300 N WESTERN AV,NaN,34.0944,-118.3125,2024
877331,241110315,8/29/24 0:00,2024-08-28,1500,11,Northeast,1123,1,420,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),...,Invest Cont,420.0,NaN,NaN,NaN,3600 BRUNSWICK AV,NaN,34.1187,-118.2604,2024
877332,242107187,3/22/24 0:00,2024-03-22,1815,21,Topanga,2145,2,624,BATTERY - SIMPLE ASSAULT,...,Adult Other,624.0,NaN,NaN,NaN,22000 GILMORE ST,NaN,34.1876,-118.6070,2024
877333,241010884,9/6/24 0:00,2024-09-05,1800,10,West Valley,1027,1,330,BURGLARY FROM VEHICLE,...,Invest Cont,330.0,NaN,NaN,NaN,6800 LOUISE AV,NaN,34.1939,-118.5098,2024


# Adding Climate Info

Climate Data From Website: https://open-meteo.com/en/docs

In [ ]:
df_2024 = df[df['Year'] == 2024].copy()
df_2024['lat_bin'] = df_2024['LAT'].round(1)
df_2024['lon_bin'] = df_2024['LON'].round(1)

grid_points = df_2024[['lat_bin', 'lon_bin']].drop_duplicates()

In [24]:
from tqdm import tqdm
import requests
import pandas as pd
import time

BASE_URL = "https://archive-api.open-meteo.com/v1/archive"

weather_records = []

for idx, row in tqdm(grid_points.iterrows(), total=len(grid_points), desc="Fetching weather by grid"):
    lat = row['lat_bin']
    lon = row['lon_bin']

    url = (
        f"{BASE_URL}?latitude={lat}&longitude={lon}"
        f"&start_date=2024-01-01&end_date=2024-12-31"
        f"&daily=temperature_2m_max,temperature_2m_min,precipitation_sum"
        f"&timezone=America%2FLos_Angeles"
    )

    try:
        r = requests.get(url, timeout=10)
        r.raise_for_status()
        data = r.json()

        daily_data = pd.DataFrame({
            'date': pd.to_datetime(data['daily']['time']),
            'lat_bin': lat,
            'lon_bin': lon,
            'temp_max': data['daily']['temperature_2m_max'],
            'temp_min': data['daily']['temperature_2m_min'],
            'precipitation': data['daily']['precipitation_sum'],
        })
        daily_data['temp_avg'] = (daily_data['temp_max'] + daily_data['temp_min']) / 2

        weather_records.append(daily_data)

    except Exception as e:
        print(f"Error fetching grid {lat},{lon}: {e}")
    time.sleep(1)

weather_df = pd.concat(weather_records, ignore_index=True)

Fetching weather by grid: 100%|██████████| 26/26 [00:41<00:00,  1.60s/it]


In [25]:
df_2024['date'] = df_2024['DATE OCC'].dt.date
df_2024['date'] = pd.to_datetime(df_2024['date'])

df_2024 = df_2024.merge(weather_df, on=['lat_bin', 'lon_bin', 'date'], how='left')
df_2024['temp_avg'] = (df_2024['temp_max'] + df_2024['temp_min']) / 2

df_2024.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,LAT,LON,Year,lat_bin,lon_bin,date,temp_max,temp_min,precipitation,temp_avg
0,242108414,4/25/24 0:00,2024-04-25,1600,21,Topanga,2146,1,420,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),...,34.2010,-118.5977,2024,34.2,-118.6,2024-04-25,18.0,9.7,0.3,13.85
1,240604934,1/21/24 0:00,2024-01-21,1510,6,Hollywood,668,2,624,BATTERY - SIMPLE ASSAULT,...,34.0944,-118.3125,2024,34.1,-118.3,2024-01-21,16.3,11.2,9.4,13.75
2,241110315,8/29/24 0:00,2024-08-28,1500,11,Northeast,1123,1,420,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),...,34.1187,-118.2604,2024,34.1,-118.3,2024-08-28,30.5,16.0,0.0,23.25
3,242107187,3/22/24 0:00,2024-03-22,1815,21,Topanga,2145,2,624,BATTERY - SIMPLE ASSAULT,...,34.1876,-118.6070,2024,34.2,-118.6,2024-03-22,20.9,10.5,0.0,15.70
4,241010884,9/6/24 0:00,2024-09-05,1800,10,West Valley,1027,1,330,BURGLARY FROM VEHICLE,...,34.1939,-118.5098,2024,34.2,-118.5,2024-09-05,44.6,23.4,0.0,34.00


# Adding Unemployment Rate

U.S. Bureau of Labor Statistics：https://www.bls.gov/developers/home.htm  &
Census Bureau: https://api.census.gov/data/2019/acs/acs5

In [17]:
import time
import json
import pandas as pd
import requests
from tqdm import tqdm
from geopy.geocoders import Nominatim
from census import Census
from us import states

BLS_API_KEY = "93c98c77c3174a458f00ae39cff06ed6"
CENSUS_API_KEY = "b44138026ba7f60eee307217b84ea86550d38f54"

c    = Census(CENSUS_API_KEY)
geo  = Nominatim(user_agent="unemp_mapper")

grid = (
    df_2024[['lat_bin','lon_bin']]
    .drop_duplicates()
    .reset_index(drop=True)
)

def lookup_county_fips(lat, lon):
    try:
        loc  = geo.reverse((lat, lon), exactly_one=True, timeout=10)
        addr = loc.raw['address']
        county = addr.get('county','').replace(" County","")
        state  = addr.get('state','')
        recs = c.acs5.state_county(
            fields=('NAME',),
            state_fips=states.lookup(state).fips,
            county_fips='*'
        )
        for rec in recs:
            if rec['NAME'].startswith(county):
                return county, state, rec['state'], rec['county']
    except Exception as e:
        print(f"Geocode/FIPS error at {lat},{lon}: {e}")
    return None, None, None, None

grid[['county','state','state_fips','county_fips']] = grid.apply(
    lambda r: lookup_county_fips(r.lat_bin, r.lon_bin),
    axis=1, result_type='expand'
)

def fetch_bls_monthly_unemp(state_fips, county_fips, year, month):
    # series_id : 'LAUCN'+<state_fips:2>+<county_fips:3>+'0000000003'
    sf = str(state_fips).zfill(2)
    cf = str(county_fips).zfill(3)
    series_id = f"LAUCN{sf}{cf}0000000003"
    payload = {
        "seriesid": [series_id],
        "startyear": str(year),
        "endyear": str(year),
        "registrationkey": BLS_API_KEY
    }
    headers = {'Content-Type': 'application/json'}
    resp = requests.post(
        "https://api.bls.gov/publicAPI/v2/timeseries/data/",
        data=json.dumps(payload),
        headers=headers,
        timeout=30
    )
    data = resp.json()
    if 'Results' not in data or 'series' not in data['Results']:
        print("BLS API error response:", data)
        return None

    for item in data['Results']['series'][0]['data']:
        if item['year']==str(year) and item['period']==f"M{month:02d}":
            return float(item['value'])
    return None

# iterate through month
records = []
for _, row in tqdm(grid.iterrows(), total=len(grid), desc="Unemp grid"):
    s_fips = row['state_fips']
    c_fips = row['county_fips']
    if not s_fips or not c_fips:
        continue
    for m in range(1,13):
        ur = fetch_bls_monthly_unemp(s_fips, c_fips, 2024, m)
        records.append({
            'lat_bin': row['lat_bin'],
            'lon_bin': row['lon_bin'],
            'year': 2024,
            'month': m,
            'unemployment_rate': ur
        })
        time.sleep(1)

df_unemp = pd.DataFrame(records)
df_unemp.head()

Unemp grid: 100%|██████████| 26/26 [06:31<00:00, 15.06s/it]


,lat_bin,lon_bin,year,month,unemployment_rate
0,34.2,-118.6,2024,1,5.6
1,34.2,-118.6,2024,2,5.3
2,34.2,-118.6,2024,3,5.3
3,34.2,-118.6,2024,4,5.0
4,34.2,-118.6,2024,5,5.4


Combine the results

In [26]:
df_2024['year']  = df_2024['DATE OCC'].dt.year
df_2024['month'] = df_2024['DATE OCC'].dt.month

df_final = df_2024.merge(
    df_unemp,
    on=['lat_bin','lon_bin','year','month'],
    how='left'
)
df_final.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,lat_bin,lon_bin,date,temp_max,temp_min,precipitation,temp_avg,year,month,unemployment_rate
0,242108414,4/25/24 0:00,2024-04-25,1600,21,Topanga,2146,1,420,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),...,34.2,-118.6,2024-04-25,18.0,9.7,0.3,13.85,2024,4,5.0
1,240604934,1/21/24 0:00,2024-01-21,1510,6,Hollywood,668,2,624,BATTERY - SIMPLE ASSAULT,...,34.1,-118.3,2024-01-21,16.3,11.2,9.4,13.75,2024,1,5.6
2,241110315,8/29/24 0:00,2024-08-28,1500,11,Northeast,1123,1,420,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),...,34.1,-118.3,2024-08-28,30.5,16.0,0.0,23.25,2024,8,6.7
3,242107187,3/22/24 0:00,2024-03-22,1815,21,Topanga,2145,2,624,BATTERY - SIMPLE ASSAULT,...,34.2,-118.6,2024-03-22,20.9,10.5,0.0,15.70,2024,3,5.3
4,241010884,9/6/24 0:00,2024-09-05,1800,10,West Valley,1027,1,330,BURGLARY FROM VEHICLE,...,34.2,-118.5,2024-09-05,44.6,23.4,0.0,34.00,2024,9,6.0


In [27]:
df_final.to_excel("Crime_2024_with_Weather_Unemp_Monthly.xlsx", index=False)